to do:
- remove hardcoding of folders/filenames
- change filessystem to pathlib
- remove unnecessary packages
- make markdown more comprehensive; include justification for design decision, such as when/when not to copy/paste
- improve comments: remove, cut down, add extra for unintuitive functions, like matrix loop at top of featuregen
- refactor slice code in generateFullFeatures()
- ???? design decision: to consolidated to into a singletest_samples_eng
- is there a more elegant way to delete object from ram than object = None
- include in readme: system limitations, dependencies, brief overview
- dependency graph on github
- reorder this notes 
- experiment fft, etc
- split pickling concsecutive/random sample pre-and post- feature generation
- edit the data visualisation to focus in on useful insights; move it to after running feature_importance on xgboost
- work on adjustment/justification for test_set data-leakage
- eliminate double -writing of xgboost params
- more explicit warnings for overwriting pickles

to do:
- remove hardcoding of folders/filenames
- change filessystem to pathlib
- remove unnecessary packages
- make markdown more comprehensive; include justification for design decision, such as when/when not to copy/paste
- improve comments: remove, cut down, add extra for unintuitive functions, like matrix loop at top of featuregen
- refactor slice code in generateFullFeatures()
- ???? design decision: to consolidated to into a singletest_samples_eng
- is there a more elegant way to delete object from ram than object = None
- include in readme: system limitations, dependencies, brief overview
- dependency graph on github
- reorder this notes 
- experiment fft, etc
- split pickling concsecutive/random sample pre-and post- feature generation
- edit the data visualisation to focus in on useful insights; move it to after running feature_importance on xgboost
- work on adjustment/justification for test_set data-leakage
- eliminate double -writing of xgboost params
- more explicit warnings for overwriting pickles

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
%matplotlib inline

#my imports
import pathlib as path
import zipfile
import pickle
import random
import time
from scipy import stats
import scipy
import statistics

from xgboost import XGBRegressor
import seaborn as sns

from sklearn.model_selection import train_test_split
import re
from pathlib import Path

# confirm code is run from folder /earthquake

In [ ]:
os.chdir('/notebooks/storage/earthquake')
os.listdir()

# download data from kaggle

In [ ]:
%%time
!kaggle competitions download LANL-Earthquake-Prediction

# unzip kaggle files

In [ ]:
%%time

train_zip='train.csv.zip'
test_zip='test.zip'
earthquake_dir='.'
test_dir='./test'

zip_ref=zipfile.ZipFile(train_zip,'r')
zip_ref.extractall(earthquake_dir)
zip_ref.close()

zip_ref=zipfile.ZipFile(test_zip,'r')
zip_ref.extractall(test_dir)
zip_ref.close()

# convert csv to dataframe

In [2]:
%%time 

train = pd.read_csv('./train.csv', dtype={"acoustic_data": np.int16, "time_to_failure": np.float32})

CPU times: user 3min 1s, sys: 13.6 s, total: 3min 14s
Wall time: 4min 50s


# store train dataframe with pickle
###### NOTE: run once then comment out

In [ ]:
# %%time

# with open('/notebooks/storage/earthquake/train.pickle','wb') as f:
#     pickle.dump(train,f,pickle.HIGHEST_PROTOCOL)

###### NOTE: code below deletes loaded train object from ram

In [ ]:
train = None

# load train dataframe

In [ ]:
%%time

with open('/notebooks/storage/earthquake/train.pickle','rb') as f:
    train = pickle.load(f)

In [ ]:
train.tail()

# filtering earthquakes to adjust mean

In [ ]:
public_mean = 4.017 
train_mean = 5.68

In [6]:
def earthquakes(train):
    ttf = train['time_to_failure'].values
    
    curr_eqtime = ttf[0]
    
    eqs = list()
    eqs = eqs + [(0,curr_eqtime)]
    
    for i in range(len(ttf)-1):
        if ttf[i+1] - ttf[i] > 0:
            eqs = eqs + [(i+1,ttf[i+1])]
            
    return eqs

In [ ]:
train['time_to_failure'].tail(4)

In [7]:
eq_list = earthquakes(train)
print(eq_list)


[(0, 1.4691), (5656574, 11.5408), (50085878, 14.1806), (104677356, 8.8567), (138772453, 12.694), (187641820, 8.0555), (218652630, 7.059), (245829585, 16.1074), (307838917, 7.9056), (338276287, 9.6371), (375377848, 11.4264), (419368880, 11.0242), (461811623, 8.8281), (495800225, 8.566), (528777115, 14.7518), (585568144, 9.4595), (621985673, 11.6186)]


In [9]:
print(len(train))

629145480


In [8]:
# with open('/notebooks/storage/earthquake/eq_list.pickle','wb') as f:
#     pickle.dump(eq_list,f,pickle.HIGHEST_PROTOCOL)

In [5]:
with open('/notebooks/storage/earthquake/eq_list.pickle','rb') as f:
    eq_list = pickle.load(f)

EOFError: Ran out of input

In [ ]:
eq_list_nums = list()

for pair in eq_list:
    eq_list_nums += [pair[1]]

In [ ]:
eq_list_nums_arr = np.array(eq_list_nums)
print(eq_list_nums_arr)

In [ ]:
onethru17 = np.arange(17)
np.mean(eq_list_nums_arr[1:16])/2

print(5.336)

eqs_chosen1 = [1,2,4,7,8,9,10,11,12,13,14,15] #5.67 #3,5,6
eqs_chosen2 = [1,2,4,5,6,7,9,10,11,14,15] #5.72 #3,8,12,13
eqs_chosen3 = [1,2,3,4,5,7,9,10,11,12,14,15] #5.69 #6,8,13
eqs_chosen4 = [1,2,3,4,5,7,9,10,11,13,14,15] #5.68 #6,8,12
# eqs_chosen5 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# eqs_chosen6 = [1,2,4,5,6,7,8,9,10,11,12,13,14,15]

count = 0
for i in eqs_chosen5:
    count += eq_list_nums_arr[i]
    
print(.5*count/len(eqs_chosen5))

In [12]:
eq0 = (0,5656574)
eq3 = (104677355,138772454)
eq5 = (187641819,218652631)
eq6 = (218652629,245829586)
eq8 = (307838916,338276288)
eq12 = (461811622,495800226)
eq13 = (495800224,528777116)
eq16 = (621985672,629145480)


In [17]:
nogo = list()
nogo += [eq0]
nogo += [eq16]
nogo1 = nogo + [eq3] + [eq5] + [eq6]
nogo2 = nogo + [eq3] + [eq8] + [eq12] + [eq13]
nogo3 = nogo + [eq6] + [eq8] + [eq13]
nogo4 = nogo + [eq6] + [eq8] + [eq12]

# generate consecutive samples

In [ ]:
def generateConsecutive(train):
    
    samples = []
    
    length = len(train)
    sample_length = 150000
    max_index = length-1
    max_start = max_index - sample_length
    
    for i in range(length//sample_length):
        
        start = i*sample_length
        end = (i+1)*sample_length
        
        samples += [[train['acoustic_data'].values[start:end],train['time_to_failure'].values[end]]]
                
    df = pd.DataFrame(samples)
        
    df = df.rename(columns = {0:'sequence',1:'ttf'})
    
    return df
    
    

In [ ]:
%%time

consecutive_samples = generateConsecutive(train)

In [ ]:
consecutive_samples.tail()

# generate N random samples

In [ ]:
def generateRandom(train,n):
    
    samples = []
    
    length = len(train)
    sample_length = 150000
    max_index = length-1
    max_start = max_index - sample_length
    
    for i in range(n):
        
        start = random.randint(0,max_start)
        end = start + sample_length
        
        samples += [[np.array(train['acoustic_data'].values[start:end]),train['time_to_failure'].values[end]]]
                
    df = pd.DataFrame(samples)
        
    df = df.rename(columns = {0:'sequence',1:'ttf'})
    
    return df

In [ ]:
def is_invalid_start(start,end,list_nogo):
    cant_start = False
    for interval in list_nogo:
        if interval[0] < start and start < interval[1]:
            cant_start = True
        if inverval[0] < end and end < interval[1]:
            cant_start = True
    
    return cant_start

In [ ]:
def generateRandom(train,n,list_nogo):
    
    samples = []
    
    length = len(train)
    sample_length = 150000
    max_index = length-1
    max_start = max_index - sample_length
    
    for i in range(n):
        
        start = random.randint(0,max_start)
        end = start + sample_length
        
        while is_invalid_start(start,end,list_nogo):
            start = random.randint(0,max_start)
            end = start + sample_length
        
        samples += [[np.array(train['acoustic_data'].values[start:end]),train['time_to_failure'].values[end]]]
                
    df = pd.DataFrame(samples)
        
    df = df.rename(columns = {0:'sequence',1:'ttf'})
    
    return df

In [ ]:
%%time

random_samples = generateRandom(train,3200)

In [ ]:
random_samples.tail()

In [ ]:
a = np.array([1,2,3,4,3,1,4,2,1,5,7,9,2,2,3])

two_largest = heapq.nlargest(2,a)
two_largest_idx = heapq.nlargest(2,range(len(a)),a.__getitem__)
print(two_largest)
print(two_largest_idx)

a = np.array([[1,2,3],[4,5,6]])
b = np.array([[2,3,4],[5,6,7]])
np.argmax(a,axis = 1)
np.append(a,b,axis = 0)

In [ ]:
print(random_samples.shape)

# full sequence features

In [ ]:
from scipy.fftpack import fft
import heapq

def generateFullFeatures(samples_df):
        
    #defining x as matrix of sequence data
    x = []
    for index,row in samples_df.iterrows():
        x += [samples_df.loc[index,'sequence']]    
    x = np.stack(x)
    x_abs = np.absolute(x)
    x_realfft = np.real(fft(x))
    
    n = 20
    
    length=150000
    
    #non-abs
#     samples_df['mean'] = np.mean(x,axis=1)
#     samples_df['median'] = np.median(x,axis=1)
    samples_df['std'] = np.std(x,axis=1)
    
#     samples_df['kurtosis'] = stats.kurtosis(x,axis=1)   
#     samples_df['m2'] = stats.moment(x,axis=1,moment=2)
#     samples_df['m3'] = stats.moment(x,axis=1,moment=3)
#     samples_df['skew'] = stats.skew(x,axis=1)
#     samples_df['variation'] = stats.variation(x,axis=1)
#     samples_df['sem'] = stats.sem(x,axis=1)

#     samples_df['iqr25_75'] = stats.iqr(x,axis=1,rng=(25,75))
    samples_df['iqr10_90'] = stats.iqr(x,axis=1,rng=(10,90))
#     samples_df['iqr5_95'] = stats.iqr(x,axis=1,rng=(5,95))
#     samples_df['iqr1_99'] = stats.iqr(x,axis=1,rng=(1,99))
    
#     nlargest = np.apply_along_axis(lambda x: heapq.nlargest(n,x),1,x_realfft)
#     nlargest_idx = np.apply_along_axis(lambda x: heapq.nlargest(n,range(len(x)),x.__getitem__),1,x_realfft)
#     for i in range(n):
#         samples_df['fft_altitude_' + str(i)] = nlargest[:,i]
#         samples_df['fft_feq_' + str(i)] = nlargest_idx[:,i]

    band1 = x_realfft[:,600:900]
    band2 = x_realfft[:,1100:1450]
    band3 = x_realfft[:,1450:1900]
    band4 = x_realfft[:,1950:2800]
    band5 = x_realfft[:,33000:42000]
    
    samples_df['band1_alt'] = np.max(band1,axis = 1)
    samples_df['band1_freq'] = np.argmax(band1,axis = 1)
    samples_df['band2_alt'] = np.max(band2,axis = 1)
    samples_df['band2_freq'] = np.argmax(band2,axis = 1)
    samples_df['band3_alt'] = np.max(band3,axis = 1)
    samples_df['band3_freq'] = np.argmax(band3,axis = 1)
    samples_df['band4_alt'] = np.max(band4,axis = 1)
    samples_df['band4_freq'] = np.argmax(band4,axis = 1)
    samples_df['band5_alt'] = np.max(band5,axis = 1)
    samples_df['band5_freq'] = np.argmax(band5,axis = 1)
    
    #np.apply_along_axis
    
    #abs
#     samples_df['mean_abs'] = np.mean(x_abs,axis=1)
    samples_df['median_abs'] = np.median(x_abs,axis=1)
    samples_df['std_abs'] = np.std(x_abs,axis=1)
    
#     samples_df['kurtosis_abs'] = stats.kurtosis(x_abs,axis=1)   
#     samples_df['m2_abs'] = stats.moment(x_abs,axis=1,moment=2)
#     samples_df['m3_abs'] = stats.moment(x_abs,axis=1,moment=3)
#     samples_df['skew_abs'] = stats.skew(x_abs,axis=1)
#     samples_df['variation_abs'] = stats.variation(x_abs,axis=1)
#     samples_df['sem_abs'] = stats.sem(x_abs,axis=1)

    
#     samples_df['iqr25_75_abs'] = stats.iqr(x_abs,axis=1,rng=(25,75))
    samples_df['iqr10_90_abs'] = stats.iqr(x_abs,axis=1,rng=(10,90))
#     samples_df['iqr5_95_abs'] = stats.iqr(x_abs,axis=1,rng=(5,95))
#     samples_df['iqr1_99_abs'] = stats.iqr(x_abs,axis=1,rng=(1,99))
    
    
    #slices
#     slices_list = [2,4]
    slices_list = [2]
    
    for slices in slices_list:

        for i in range(slices):

            suffix = '_'+str(slices)+'_'+str(i+1)

            #create same as above, but for first half and second half
            
            # STYLE: DEFINE sliced_x at top of inner loop to make code prettier

            current_slice = x[:,i*(length//slices):(i+1)*(length//slices)]
            current_slice_abs = x_abs[:,i*(length//slices):(i+1)*(length//slices)]
            current_slice_realfft = np.real(fft(current_slice))
            
            #non-abs
#             samples_df['mean'+suffix] = np.mean(current_slice,axis=1)
#             samples_df['median'+suffix] = np.median(current_slice,axis=1)
            samples_df['std'+suffix] = np.std(current_slice,axis=1)

#             samples_df['kurtosis'+suffix] = stats.kurtosis(current_slice,axis=1)
#             samples_df['m2'+suffix] = stats.moment(current_slice,axis=1,moment=2)
#             samples_df['m3'+suffix] = stats.moment(current_slice,axis=1,moment=3)
#             samples_df['skew'+suffix] = stats.skew(current_slice,axis=1)
#             samples_df['variation'+suffix] = stats.variation(current_slice,axis=1)
#             samples_df['sem'+suffix] = stats.sem(current_slice,axis=1)

#             samples_df['iqr25_75'+suffix] = stats.iqr(current_slice,axis=1,rng=(25,75))
            samples_df['iqr10_90'+suffix] = stats.iqr(current_slice,axis=1,rng=(10,90))
#             samples_df['iqr5_95'+suffix] = stats.iqr(current_slice,axis=1,rng=(5,95))
#             samples_df['iqr1_99'+suffix] = stats.iqr(current_slice,axis=1,rng=(1,99))
            
#             nlargest_slice = np.apply_along_axis(lambda x: heapq.nlargest(n,x),1,current_slice_realfft)
#             nlargest_idx_slice = np.apply_along_axis(lambda x: heapq.nlargest(n,range(len(x)),x.__getitem__),1,current_slice_realfft)
#             for i in range(n):
#                 samples_df['fft_altitude_' + str(i)] = nlargest_slice[:,i]
#                 samples_df['fft_feq_' + str(i)] = nlargest_idx_slice[:,i]

            band1_slice = x_realfft[:,600:900]
            band2_slice = x_realfft[:,1100:1450]
            band3_slice = x_realfft[:,1450:1900]
            band4_slice = x_realfft[:,1950:2800]
            band5_slice = x_realfft[:,33000:42000]

            samples_df['band1_alt' + suffix] = np.max(band1_slice,axis = 1)
            samples_df['band1_freq' + suffix] = np.argmax(band1_slice,axis = 1)
            samples_df['band2_alt' + suffix] = np.max(band2_slice,axis = 1)
            samples_df['band2_freq' + suffix] = np.argmax(band2_slice,axis = 1)
            samples_df['band3_alt' + suffix] = np.max(band3_slice,axis = 1)
            samples_df['band3_freq' + suffix] = np.argmax(band3_slice,axis = 1)
            samples_df['band4_alt' + suffix] = np.max(band4_slice,axis = 1)
            samples_df['band4_freq' + suffix] = np.argmax(band4_slice,axis = 1)
            samples_df['band5_alt' + suffix] = np.max(band5_slice,axis = 1)
            samples_df['band5_freq' + suffix] = np.argmax(band5_slice,axis = 1)            


            #abs
#             samples_df['mean_abs'+suffix] = np.mean(current_slice_abs,axis=1)
            samples_df['median_abs'+suffix] = np.median(current_slice_abs,axis=1)
            samples_df['std_abs'+suffix] = np.std(current_slice_abs,axis=1)

#             samples_df['kurtosis_abs'+suffix] = stats.kurtosis(current_slice_abs,axis=1)
#             samples_df['m2_abs'+suffix] = stats.moment(current_slice_abs,axis=1,moment=2)
#             samples_df['m3_abs'+suffix] = stats.moment(current_slice_abs,axis=1,moment=3)
#             samples_df['skew_abs'+suffix] = stats.skew(current_slice_abs,axis=1)
#             samples_df['variation_abs'+suffix] = stats.variation(current_slice_abs,axis=1)
#             samples_df['sem_abs'+suffix] = stats.sem(current_slice_abs,axis=1)


#             samples_df['iqr25_75_abs'+suffix] = stats.iqr(current_slice_abs,axis=1,rng=(25,75))
            samples_df['iqr10_90_abs'+suffix] = stats.iqr(current_slice_abs,axis=1,rng=(10,90))
#             samples_df['iqr5_95_abs'+suffix] = stats.iqr(current_slice_abs,axis=1,rng=(5,95))
#             samples_df['iqr1_99_abs'+suffix] = stats.iqr(current_slice_abs,axis=1,rng=(1,99))            
            

In [ ]:
%%time
# all_samples = random_samples.append(consecutive_samples)
generateFullFeatures(random_samples)

In [ ]:
random_samples.tail()

In [ ]:
random_samples['band5_freq']

# pickle samples

In [ ]:
%%time

with open('/notebooks/storage/earthquake/consecutive_samples.pickle','wb') as f:
    pickle.dump(consecutive_samples,f,pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/notebooks/storage/earthquake/random_samples.pickle','wb') as f:
    pickle.dump(random_samples,f,pickle.HIGHEST_PROTOCOL)

In [ ]:
consecutive_samples = None

# load samples

In [ ]:
%%time

with open('/notebooks/storage/earthquake/consecutive_samples.pickle','rb') as f:
    consecutive_samples = pickle.load(f)

In [ ]:
consecutive_samples.tail()

# visualize data

In [ ]:
random_samples.plot(x='iqr1_99_abs',y='ttf',kind='scatter',logx=True)

In [ ]:
random_samples.plot.hexbin(x='mean',y='ttf',gridsize=10)

In [ ]:
sns.kdeplot(consecutive_samples['ttf'],consecutive_samples['mean'])

In [ ]:
sns.kdeplot(consecutive_samples.ttf)


In [ ]:
sns.jointplot(x='iqr1_99_abs',y='ttf',data=consecutive_samples,kind='hex',gridsize=10)

In [ ]:
sns.pairplot(consecutive_samples[['ttf','iqr1_99_abs','iqr5_95_abs','iqr10_90_abs']])

In [ ]:
ex1 = fft(random_samples['sequence'].values[10][:5000])
xvals = np.arange(0,5000)
sns.jointplot(x = xvals,y = ex1)

In [ ]:
# look into plotly for interactive plots, and ggplot for grammar of graphics

# split data into train/validation

In [ ]:
public_mean = 4.017 
train_mean = 5.68
mul_factor = public_mean/train_mean


X = random_samples.drop(labels=['ttf','sequence'],axis=1)
Y = random_samples['ttf']/mul_factor

x_train, x_val, y_train, y_val = train_test_split(X,Y,test_size=.1,random_state=0,shuffle=False)

In [ ]:
x_val.shape

# XGBoost

In [ ]:
%%time


n_estimators = 10000
learning_rate = .001
n_jobs = 8

early_stopping_rounds = 20
eval_set = [(x_val,y_val)]
verbose = True

model_xgb = XGBRegressor(
    n_estimators = n_estimators, 
    learning_rate = learning_rate, 
    n_job = n_jobs
)

model_xgb.fit(x_train,y_train,
              early_stopping_rounds= early_stopping_rounds,
              eval_set = eval_set,
              eval_metric = 'mae',
              verbose = verbose
             )

In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = model_xgb.predict(x_val)
print('mae : '+str(mean_absolute_error(predictions,y_val)))

# OVERWRITE pickle model

In [ ]:
%%time

with open('/notebooks/storage/earthquake/model_xgb.pickle','wb') as f:
    pickle.dump(model_xgb,f,pickle.HIGHEST_PROTOCOL)

In [ ]:
model_xgb = None

# LOAD pickled model

In [ ]:
%%time

with open('/notebooks/storage/earthquake/model_xgb.pickle','rb') as f:
    model_xgb = pickle.load(f)

# create submission
#### start with creating a dataframe identical in format to consecutive/random samples above

In [ ]:
%%time

os.chdir('/notebooks/storage/earthquake/test')
# os.remove("submission.csv")
test_samples = pd.DataFrame(columns=['sequence','ttf'])

files = [x for x in os.listdir() if x[-4:] == '.csv']

for i in range(len(files)):
    
    test_file = files[i]
    
    temp_df = pd.read_csv('./'+test_file,engine='python')
        
    test_samples.loc[i,'sequence'] = np.array(temp_df['acoustic_data'].values[:])

In [ ]:
test_samples.tail()

# OVERWRITE pickle test sample df

In [ ]:
# %%time

# with open('/notebooks/storage/earthquake/test_samples.pickle','wb') as f:
#     pickle.dump(test_samples,f,pickle.HIGHEST_PROTOCOL)

In [ ]:
test_samples = None

# LOAD pickled test sample df

In [ ]:
%%time

with open('/notebooks/storage/earthquake/test_samples.pickle','rb') as f:
    test_samples = pickle.load(f)

# add features

In [ ]:
%time

generateFullFeatures(test_samples)

In [ ]:
test_samples.tail()

# pickle engineered sample

In [ ]:
# %%time

# with open('/notebooks/storage/earthquake/test_samples_eng.pickle','wb') as f:
#     pickle.dump(test_samples,f,pickle.HIGHEST_PROTOCOL)

In [ ]:
test_samples = None

# load pickled engineered sample

In [ ]:
%%time

with open('/notebooks/storage/earthquake/test_samples_eng.pickle','rb') as f:
    test_samples_eng = pickle.load(f)

In [ ]:
test_samples_eng.shape

# make predictions/submit

In [ ]:
test_x = test_samples_eng.drop(columns=['sequence','ttf'])

y_pred = model_xgb.predict(test_x)

files = [x for x in os.listdir('/notebooks/storage/earthquake/test') if x[-4:] == '.csv']


In [ ]:
y_pred.shape

In [ ]:
submission_df = pd.DataFrame(columns=['seg_id','time_to_failure'])

In [ ]:
submission_df['seg_id'] = pd.Series([i[:-4] for i in files])
submission_df.shape

In [ ]:
submission_df['time_to_failure'] = y_pred

In [ ]:
submission_df.tail()

In [ ]:
submission_df.to_csv('/notebooks/storage/earthquake/submission.csv',index=False)